In [13]:
import os, sys
import numpy as np
import dask
import xarray as xr
import xgcm

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

In [14]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(20)

In [15]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [16]:
client

Client Scheduler: tcp://10.148.1.35:37720 Dashboard: http://10.148.1.35:45705/status,Cluster Workers: 8 Cores: 32 Memory: 400.00 GB


--- 

# standard data layout: (face, j, i), 1 file per time (for movies)

__________

In [42]:
V = ['Eta']

out_dir = '/home1/scratch/xyu/mit_zarr/'  
Nt = 1

for v in V:

    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    it = np.arange(1512,2232).astype(int)*Nt # tmp
    it = np.arange(1512,1517).astype(int)*Nt # tmp

    assert it[-1]+Nt<time.size
    #
    p = 'C'
    vdrop = ['XC','YC','Depth','rA','CS','SN','dtime']
    if v is 'SSU':
        p = 'W'
        vdrop = ['dxC','dyG','rAw']         
    elif v is 'SSV':
        p = 'S'
        vdrop = ['dxG','dyC','rAs']        
        #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
        #
        
    for i, t in enumerate(it):
        #
        file_out = out_dir+'%s_t%04d.zarr'%(v,i)
        dv = ds[v].isel(time=slice(t,t+Nt)) 
        dv = dv.drop(vdrop).to_dataset()
        dv.to_zarr(file_out, mode='w')     

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


_____
# For each face

In [7]:
V = ['Eta']
out_dir = '/home1/scratch/xyu/mit_zarr/'  
face_all = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] # some key faces

for face in face_all :
    
    for v in V:

    #
        data_dir = root_data_dir+v+'/'
        iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
        p = 'C'
        if v is 'SSU':
            p = 'W'
        elif v is 'SSV':
            p = 'S'
    #
        ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #
    # should store grid data independantly in a single file
        ds = ds.drop(['XC','YC','CS','SN','rA','Depth'])
    #
    #ds = ds.chunk({'face': 1})
    #
        dv = ds[v].isel(time=slice(1512,2232),face=face)
        dv = dv.chunk({'time': 1,'i':4320,'j':4320})

        dv = dv.to_dataset()
        file_out = out_dir+'/%s_f%02d.zarr'%(v,face)
        %time dv.to_zarr(file_out, mode='w')                    

        dsize = getsize(file_out)
        print('   data is %.1fGB ' %(dsize/1e9))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


CPU times: user 4.41 s, sys: 388 ms, total: 4.8 s
Wall time: 26.1 s
   data is 13.8GB 


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


CPU times: user 4.5 s, sys: 276 ms, total: 4.78 s
Wall time: 29.6 s
   data is 29.1GB 


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


...

CPU times: user 4.48 s, sys: 272 ms, total: 4.75 s
Wall time: 25.7 s
   data is 18.2GB 


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


CPU times: user 4.25 s, sys: 376 ms, total: 4.62 s
Wall time: 26 s
   data is 29.1GB 


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


CPU times: user 4.34 s, sys: 296 ms, total: 4.64 s
Wall time: 25.4 s
   data is 16.5GB 


In [56]:
data_dir = '/home1/scratch/xyu/mit_zarr/Eta/'

%time D = [xr.open_zarr(data_dir+'Eta_t%04d.zarr'%(t), decode_cf=False) for t in range(720)]
ds_Eta = xr.concat(D, dim='time')
vdrop = ['CS','SN','dtime']
ds_Eta = ds_Eta.drop(vdrop)

grid = xr.open_zarr('/home1/scratch/xyu/mit_grid.zarr')

CPU times: user 13.1 s, sys: 4.94 s, total: 18 s
Wall time: 26.4 s


In [57]:
ds = xr.merge([ds_Eta, grid])
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, time: 720)
Coordinates:
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 5.713e+06 5.717e+06 ...
    CS       (face, j, i) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 432, 432)>
    Depth    (face, j, i) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 432, 432)>
    SN       (face, j, i) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 432, 432)>
    XC       (face, j, i) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 432, 432)>
    XG       (face, j_g, i_g) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 432, 432)>
    YC       (face, j, i) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 432, 432)>
    YG       (face, j_g, i_g) f

In [59]:
cluster.scheduler.close()

<Future finished result=None>